In [1]:
!pip install grpcio tensorflow-serving-api


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2, prediction_service_pb2_grpc
import numpy as np

In [13]:
host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)


In [6]:
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import tensorflow.lite as tflite

In [10]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x
def np_to_protobuf(data):
    return tf.make_tensor_proto(data,shape = data.shape)

In [8]:
pants_url = 'https://bit.ly/pants-test'
hat_url = 'https://bit.ly/hat-test'
response = requests.get(hat_url)
if response.status_code != 200:
    raise ValueError(f"Failed to download image. Status code: {response.status_code}")
with Image.open(BytesIO(response.content)) as img:
    img = img.resize((299, 299), Image.NEAREST)
x = np.array(img, dtype='float32')
X = np.array([x])
X= preprocess_input(X)

In [15]:
pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [16]:
pb_response = stub.Predict(pb_request,timeout = 20.0)

In [18]:

classes = ['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']
raw_output = pb_response.outputs["dense_7"].float_val
output_array = np.array(raw_output)
pred_index = np.argmax(output_array)
predicted_class = classes[pred_index].capitalize()
predicted_score = output_array[pred_index]

print(f"Predicted class: {predicted_class} (score: {predicted_score:.4f})")


Predicted class: Hat (score: 2.5003)
